In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
csv_file = "datasets/movies_bechdel.csv"
bechdel_df = pd.read_csv(csv_file)
bechdel_df.head()

,year,imdb,title,test,clean_test,binary,budget,domgross,intgross,code,budget_2013$,domgross_2013$,intgross_2013$,period code,decade code
0,2013,tt1711425,21 & Over,notalk,notalk,FAIL,13000000,25682380.0,42195766.0,2013FAIL,13000000,25682380.0,42195766.0,1.0,1.0
1,2012,tt1343727,Dredd 3D,ok-disagree,ok,PASS,45000000,13414714.0,40868994.0,2012PASS,45658735,13611086.0,41467257.0,1.0,1.0
2,2013,tt2024544,12 Years a Slave,notalk-disagree,notalk,FAIL,20000000,53107035.0,158607035.0,2013FAIL,20000000,53107035.0,158607035.0,1.0,1.0
3,2013,tt1272878,2 Guns,notalk,notalk,FAIL,61000000,75612460.0,132493015.0,2013FAIL,61000000,75612460.0,132493015.0,1.0,1.0
4,2013,tt0453562,42,men,men,FAIL,40000000,95020213.0,95020213.0,2013FAIL,40000000,95020213.0,95020213.0,1.0,1.0


In [27]:
new_bechdel_df = bechdel_df[["imdb","title","binary"]].copy()

In [28]:
new_bechdel_df = new_bechdel_df.rename(columns = {
    "imdb":"imdb_id",
    "binary":"pass_fail"
})

In [33]:
new_bechdel_df['title'] = new_bechdel_df['title'].astype('str')
new_bechdel_df["title"] = new_bechdel_df["title"].apply\
(lambda x: x.lower())
new_bechdel_df.head()

,imdb_id,title,pass_fail
0,tt1711425,21 & over,FAIL
1,tt1343727,dredd 3d,PASS
2,tt2024544,12 years a slave,FAIL
3,tt1272878,2 guns,FAIL
4,tt0453562,42,FAIL


In [34]:
original_budgets = bechdel_df[["imdb", "title", "budget_2013$", "domgross", "intgross"]]
original_budgets = original_budgets.rename(columns={
    "imdb":"imdb_id",
    "budget_2013$":"original_budget",
    "domgross":"domestic_gross",
    "intgross":"international_gross"
})
original_budgets["title"] = original_budgets["title"].apply\
(lambda x: x.lower())

original_budgets.head()

,imdb_id,title,original_budget,domestic_gross,international_gross
0,tt1711425,21 & over,13000000,25682380.0,42195766.0
1,tt1343727,dredd 3d,45658735,13414714.0,40868994.0
2,tt2024544,12 years a slave,20000000,53107035.0,158607035.0
3,tt1272878,2 guns,61000000,75612460.0,132493015.0
4,tt0453562,42,40000000,95020213.0,95020213.0


In [36]:
adj_2013_budgets = bechdel_df[["imdb", "title", "budget", "domgross_2013$", "intgross_2013$"]]
adj_2013_budgets = adj_2013_budgets.rename(columns={
    "imdb":"imdb_id",
    "budget":"adj_budget",
    "domgross_2013$":"adj_dom_gross",
    "intgross_2013$":"adj_int_gross"
})
adj_2013_budgets["title"] = adj_2013_budgets["title"].apply\
(lambda x: x.lower())

adj_2013_budgets.head()

,imdb_id,title,adj_budget,adj_dom_gross,adj_int_gross
0,tt1711425,21 & over,13000000,25682380.0,42195766.0
1,tt1343727,dredd 3d,45000000,13611086.0,41467257.0
2,tt2024544,12 years a slave,20000000,53107035.0,158607035.0
3,tt1272878,2 guns,61000000,75612460.0,132493015.0
4,tt0453562,42,40000000,95020213.0,95020213.0


In [37]:
#Connecting to local database
rds_connection_string = "postgres:postgres@localhost:5432/bechdel_netflix_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [38]:
#Checking for tables
engine.table_names()

['bechdel_movies', 'original_budgets', 'adjusted_budgets_2013']

In [39]:
#Loading dataframes into database
new_bechdel_df.to_sql(name='bechdel_movies', con=engine, if_exists='append', index=False)
adj_2013_budgets.to_sql(name='adjusted_budgets_2013', con=engine, if_exists='append', index=False)
original_budgets.to_sql(name='original_budgets', con=engine, if_exists='append', index=False)

In [42]:
#Confirming database additions
pd.read_sql_query('select * from adjusted_budgets_2013', con=engine).head()

,imdb_id,title,adj_budget,adj_dom_gross,adj_int_gross
0,tt1711425,21 & over,13000000.0,25682380.0,42195766.0
1,tt1343727,dredd 3d,45000000.0,13611086.0,41467257.0
2,tt2024544,12 years a slave,20000000.0,53107035.0,158607035.0
3,tt1272878,2 guns,61000000.0,75612460.0,132493015.0
4,tt0453562,42,40000000.0,95020213.0,95020213.0
